In [ ]:
import pickle
import omegaconf
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

project_name = 'full_fine_tuning_5epochs_final1'

datas=( 'uc-merced-land-use-dataset',  'stanford_cars', 'caltech101', 'fgvc-aircraft-2013b', 'dtd', 'flowers-102', 'oxford-iiit-pet'  ) 

losses=( 'TRADES_v2', 'CLASSIC_AT' ) #  

backbones=(
  'convnext_tiny', 'robust_convnext_tiny', 'convnext_tiny.fb_in22k', 
  'deit_small_patch16_224.fb_in1k', 'robust_deit_small_patch16_224',
  'convnext_base', 'convnext_base.fb_in22k', 'robust_convnext_base', 
  'convnext_base.clip_laion2b', 'convnext_base.clip_laion2b_augreg',
  'vit_base_patch16_224.augreg_in1k', 'vit_base_patch16_224.augreg_in21k',
  'vit_base_patch16_224.dino', 'vit_base_patch16_224.mae', 'vit_base_patch16_224.orig_in21k',
  'vit_base_patch16_224.sam_in1k', 'vit_base_patch16_224_miil.in21k', 'robust_vit_base_patch16_224' )  

import pickle
import numpy as np

final_data = []

for backbone in backbones:
    for loss in losses:
        for data in datas:

            try:

                name ='{}_{}_{}'.format(backbone, data, loss)
                with open('./results/results_{}_{}_{}_{}.pkl'.format(project_name, backbone, data, loss), 'rb') as f:
                    saved_data = pickle.load(f)
                    result = saved_data[name]["statistics"]
                    result['backbone'] = backbone
                    result['dataset'] = data
                    result['loss_function'] = loss    

                    print(name, saved_data[name]["statistics"])
            except:
                result = {}
                result['backbone'] = backbone
                result['dataset'] = data
                result['loss_function'] = loss   
                result['clean_acc'] = np.nan
                result['robust_acc'] = np.nan 
                # print('not available')

            final_data.append(result)

/home/mheuillet/Desktop/robust_training/.venv2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-27 11:03:54,026	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-02-27 11:03:54,397	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


convnext_tiny_uc-merced-land-use-dataset_TRADES_v2 {'clean_acc': 0.44047619047619047, 'Linf_acc': 0.0, 'L2_acc': None, 'L1_acc': None, 'backbone': 'convnext_tiny', 'dataset': 'uc-merced-land-use-dataset', 'loss_function': 'TRADES_v2'}
convnext_tiny_kvasir-dataset_TRADES_v2 {'clean_acc': 0.27625, 'Linf_acc': 0.0, 'L2_acc': None, 'L1_acc': None, 'backbone': 'convnext_tiny', 'dataset': 'kvasir-dataset', 'loss_function': 'TRADES_v2'}
convnext_tiny_stanford_cars_TRADES_v2 {'clean_acc': 0.3606666666666667, 'robust_acc': 0.004, 'backbone': 'convnext_tiny', 'dataset': 'stanford_cars', 'loss_function': 'TRADES_v2'}
convnext_tiny_caltech101_TRADES_v2 {'clean_acc': 0.9466666666666667, 'robust_acc': 0.8073333333333333, 'backbone': 'convnext_tiny', 'dataset': 'caltech101', 'loss_function': 'TRADES_v2'}
convnext_tiny_fgvc-aircraft-2013b_TRADES_v2 {'clean_acc': 0.14466666666666667, 'robust_acc': 0.0006666666666666666, 'backbone': 'convnext_tiny', 'dataset': 'fgvc-aircraft-2013b', 'loss_function': 'TR

In [2]:
import pandas as pd

df = pd.DataFrame(final_data)
df

,clean_acc,Linf_acc,L2_acc,L1_acc,backbone,dataset,loss_function,robust_acc,common_acc
0,0.440476,0.000000,NaN,NaN,convnext_tiny,uc-merced-land-use-dataset,TRADES_v2,NaN,NaN
1,0.276250,0.000000,NaN,NaN,convnext_tiny,kvasir-dataset,TRADES_v2,NaN,NaN
2,0.360667,NaN,NaN,NaN,convnext_tiny,stanford_cars,TRADES_v2,0.004000,NaN
3,0.946667,NaN,NaN,NaN,convnext_tiny,caltech101,TRADES_v2,0.807333,NaN
4,0.144667,NaN,NaN,NaN,convnext_tiny,fgvc-aircraft-2013b,TRADES_v2,0.000667,NaN
...,...,...,...,...,...,...,...,...,...
283,0.125333,0.035333,NaN,NaN,robust_vit_base_patch16_224,caltech101,CLASSIC_AT,NaN,0.022727
284,0.012000,0.004000,NaN,NaN,robust_vit_base_patch16_224,fgvc-aircraft-2013b,CLASSIC_AT,NaN,NaN
285,0.173333,0.117333,NaN,NaN,robust_vit_base_patch16_224,dtd,CLASSIC_AT,NaN,NaN
286,0.064000,0.038000,NaN,NaN,robust_vit_base_patch16_224,flowers-102,CLASSIC_AT,NaN,NaN


In [3]:

# Define the valid condition for each row:
#   - either both are not NaN,
#   - or exactly one is NaN (and then the other must be not NaN)
valid_condition = (
    (df['robust_acc'].isna() & df['Linf_acc'].notna()) | 
    (df['Linf_acc'].isna() & df['robust_acc'].notna()) | 
    (df['robust_acc'].isna() & df['Linf_acc'].isna())
)

if valid_condition.all():
    print("Condition holds: If one column is NaN, the other is necessarily not NaN.")
else:
    print("Condition does NOT hold. The following rows do not meet the requirement:")

# set(df[~valid_condition]['dataset'])

df[~valid_condition]


Condition holds: If one column is NaN, the other is necessarily not NaN.


,clean_acc,Linf_acc,L2_acc,L1_acc,backbone,dataset,loss_function,robust_acc,common_acc


In [4]:
df[ df['Linf_acc'].notna() ]

,clean_acc,Linf_acc,L2_acc,L1_acc,backbone,dataset,loss_function,robust_acc,common_acc
0,0.440476,0.000000,NaN,NaN,convnext_tiny,uc-merced-land-use-dataset,TRADES_v2,NaN,NaN
1,0.276250,0.000000,NaN,NaN,convnext_tiny,kvasir-dataset,TRADES_v2,NaN,NaN
8,0.047619,0.047619,NaN,NaN,convnext_tiny,uc-merced-land-use-dataset,CLASSIC_AT,NaN,NaN
9,0.382500,0.000000,NaN,NaN,convnext_tiny,kvasir-dataset,CLASSIC_AT,NaN,NaN
16,0.900000,0.642857,NaN,NaN,robust_convnext_tiny,uc-merced-land-use-dataset,TRADES_v2,NaN,NaN
24,0.914286,0.733333,NaN,NaN,robust_convnext_tiny,uc-merced-land-use-dataset,CLASSIC_AT,NaN,NaN
32,0.800000,0.111905,NaN,NaN,convnext_tiny.fb_in22k,uc-merced-land-use-dataset,TRADES_v2,NaN,NaN
40,0.047619,0.047619,NaN,NaN,convnext_tiny.fb_in22k,uc-merced-land-use-dataset,CLASSIC_AT,NaN,NaN
41,0.680000,0.000000,NaN,NaN,convnext_tiny.fb_in22k,kvasir-dataset,CLASSIC_AT,NaN,NaN
48,0.740476,0.059524,NaN,NaN,deit_small_patch16_224.fb_in1k,uc-merced-land-use-dataset,TRADES_v2,NaN,NaN


In [5]:
df['Linf_acc'] = df['robust_acc'].combine_first(df['Linf_acc'])
# df['Linf_acc'] = df['Linf_acc'].combine_first(df['robust_acc'])

In [6]:
data_updated = df.copy()

# Combine clean_acc and robust_acc as independent metrics for ranking
data_updated['total_acc'] = data_updated['clean_acc'] + data_updated['Linf_acc']  # Sum of scores per dataset

# Aggregate total_acc across all datasets for each backbone + loss combination
aggregated_data_no_avg = data_updated.groupby(['backbone', 'loss_function']).agg(
    total_sum=('total_acc', 'sum'),  # Total sum of all scores across datasets
    total_geom_mean=('total_acc', lambda x: np.prod(x) ** (1 / len(x))),  # Geometric mean
).reset_index()

# Rank based on these combined scores
aggregated_data_no_avg['rank_total_sum'] = aggregated_data_no_avg['total_sum'].rank(ascending=False)
aggregated_data_no_avg['rank_total_geom_mean'] = aggregated_data_no_avg['total_geom_mean'].rank(ascending=False)

# Sort by sum ranking for presentation
ranked_aggregated_data_no_avg = aggregated_data_no_avg.sort_values(by='rank_total_sum')
ranked_aggregated_data_no_avg

# tools.display_dataframe_to_user(name="Ranked Backbone and Loss Combinations Without Averaging", dataframe=ranked_aggregated_data_no_avg)

,backbone,loss_function,total_sum,total_geom_mean,rank_total_sum,rank_total_geom_mean
1,convnext_base,TRADES_v2,9.540667,1.404318,1.0,1.0
4,convnext_base.clip_laion2b_augreg,CLASSIC_AT,9.298667,1.377144,2.0,2.0
2,convnext_base.clip_laion2b,CLASSIC_AT,9.226000,1.367127,3.0,4.0
11,convnext_tiny.fb_in22k,TRADES_v2,9.211333,1.367145,4.0,3.0
17,robust_convnext_tiny,TRADES_v2,8.998667,1.342688,5.0,5.0
31,vit_base_patch16_224.orig_in21k,TRADES_v2,8.883333,1.324775,6.0,7.0
33,vit_base_patch16_224.sam_in1k,TRADES_v2,8.826667,1.321060,7.0,8.0
0,convnext_base,CLASSIC_AT,8.818000,1.300676,8.0,11.0
6,convnext_base.fb_in22k,CLASSIC_AT,8.730667,1.289134,9.0,13.0
9,convnext_tiny,TRADES_v2,8.730000,1.302082,10.0,10.0


In [6]:
# Pivot the table to group by 'backbone' and 'loss_function' in rows and create a hierarchical structure with 'dataset' as column index
grouped_df = df.pivot_table(
    index=['backbone', 'loss_function'], 
    columns='dataset', 
    values=[  'clean_acc', 'Linf_acc'  ], #
    dropna=False )

grouped_df #.to_csv('results_acc.csv' )

Linf_acc            \
dataset                                         caltech101       dtd   
backbone                          loss_function                        
convnext_base                     CLASSIC_AT      0.924667  0.460667   
                                  TRADES_v2       0.922000  0.355333   
convnext_base.clip_laion2b        CLASSIC_AT      0.876667  0.331333   
                                  TRADES_v2       0.833333  0.245333   
convnext_base.clip_laion2b_augreg CLASSIC_AT      0.870000  0.346000   
                                  TRADES_v2       0.836000  0.243333   
convnext_base.fb_in22k            CLASSIC_AT      0.928000  0.251833   
                                  TRADES_v2       0.924667  0.339333   
convnext_tiny                     CLASSIC_AT      0.845333  0.061500   
                                  TRADES_v2       0.807333  0.090833   
convnext_tiny.fb_in22k            CLASSIC_AT      0.891333  0.296889   
                                  TRADES_v2       0.892000  0.224000   
deit_small_patch16_224.fb_in1k    CLASSIC_AT      0.841333  0.312667   
                                  TRADES_v2       0.818667  0.224000   
robust_convnext_base              CLASSIC_AT      0.929333  0.411333   
                                  TRADES_v2       0.931333  0.341333   
robust_convnext_tiny              CLASSIC_AT      0.921333  0.367333   
                                  TRADES_v2       0.899333  0.260667   
robust_deit_small_patch16_224     CLASSIC_AT      0.904667  0.338667   
                                  TRADES_v2       0.890000  0.331333   
robust_vit_base_patch16_224       CLASSIC_AT      0.035333  0.117333   
                                  TRADES_v2       0.606667  0.214667   
vit_base_patch16_224.augreg_in1k  CLASSIC_AT      0.828667  0.289333   
                                  TRADES_v2       0.814000  0.239333   
vit_base_patch16_224.augreg_in21k CLASSIC_AT      0.788667  0.277000   
                                  TRADES_v2       0.848000  0.046833   
vit_base_patch16_224.dino         CLASSIC_AT      0.838667  0.296667   
                                  TRADES_v2       0.824000  0.257333   
vit_base_patch16_224.mae          CLASSIC_AT      0.694667  0.192667   
                                  TRADES_v2       0.633333  0.142000   
vit_base_patch16_224.orig_in21k   CLASSIC_AT      0.804000  0.335333   
                                  TRADES_v2       0.818000  0.215333   
vit_base_patch16_224.sam_in1k     CLASSIC_AT      0.818000  0.322667   
                                  TRADES_v2       0.839333  0.281333   
vit_base_patch16_224_miil.in21k   CLASSIC_AT      0.252667  0.189333   
                                  TRADES_v2       0.536667  0.120667   

                                                                     \
dataset                                         fgvc-aircraft-2013b   
backbone                          loss_function                       
convnext_base                     CLASSIC_AT               0.006500   
                                  TRADES_v2                0.088000   
convnext_base.clip_laion2b        CLASSIC_AT               0.100000   
                                  TRADES_v2                0.179333   
convnext_base.clip_laion2b_augreg CLASSIC_AT               0.106000   
                                  TRADES_v2                0.014667   
convnext_base.fb_in22k            CLASSIC_AT               0.004667   
                                  TRADES_v2                0.134667   
convnext_tiny                     CLASSIC_AT               0.000000   
                                  TRADES_v2                0.000667   
convnext_tiny.fb_in22k            CLASSIC_AT               0.003333   
                                  TRADES_v2                0.083333   
deit_small_patch16_224.fb_in1k    CLASSIC_AT               0.008000   
                                  TRADES_v2                0.015333   
robust_convnext_base              

In [26]:
import pandas as pd
import numpy as np
import plotly.express as px


df_new_cleaned = df #.drop(columns=['Unnamed: 0'])
name =  'TRADES_v2' #
df_new_cleaned = df_new_cleaned[ df_new_cleaned.loss_function != name ]

# Assuming df_new_cleaned already exists and contains all necessary columns
# Group by backbone and loss function, compute the mean and standard deviation for clean and robust accuracy
aggregated_metrics = df_new_cleaned.groupby(['backbone', 'loss_function']).agg(
    Mean_Clean_Acc=('clean_acc', 'mean'),
    Std_Clean_Acc=('clean_acc', 'std'),
    Mean_Robust_Acc=('robust_acc', 'mean'),
    Std_Robust_Acc=('robust_acc', 'std')
).reset_index()

# Classify architectures into categories
aggregated_metrics['Architecture'] = aggregated_metrics['backbone'].apply(
    lambda name: 'ConvNext' if 'convnext' in name.lower() else ('ViT/DeiT' if 'deit' in name.lower() or 'vit' in name.lower() else 'Other')
)

# Assign marker shapes based on loss function
aggregated_metrics['Marker_Shape'] = aggregated_metrics['loss_function'].apply(
    lambda loss: 'circle' if loss == 'TRADES_v2' else ('square' if loss == 'CLASSIC_AT' else 'diamond')
)

# Identify Pareto front
x = aggregated_metrics['Mean_Clean_Acc']
y = aggregated_metrics['Mean_Robust_Acc']
is_pareto = np.array([
    not any((y[j] >= y[i] and x[j] >= x[i]) for j in range(len(x)) if i != j)
    for i in range(len(x))
])
pareto_points = aggregated_metrics[is_pareto]

# Add a mapping for loss functions to valid Plotly symbols
symbol_mapping = {
    'TRADES_v2': 'circle',
    'CLASSIC_AT': 'square'
}

# Update the scatter plot with proper legend entries for loss functions
fig = px.scatter(
    aggregated_metrics,
    x='Mean_Clean_Acc',
    y='Mean_Robust_Acc',
    color='Architecture',         # Color for architecture
    symbol='loss_function',       # Symbol for loss function
    # title='Backbone Performance: Clean vs. Robust Accuracy',
    labels={
        'Mean_Clean_Acc': 'Mean Clean Accuracy',
        'Mean_Robust_Acc': 'Mean Robust Accuracy',
        'loss_function': 'Loss Function'  # Custom label for legend
    },
    color_discrete_map={
        'ConvNext': '#1f77b4',  # Blue for ConvNext
        'ViT/DeiT': '#ff7f0e',  # Orange for ViT/DeiT
        'Other': '#2ca02c'      # Green for Other architectures
    },
    symbol_map=symbol_mapping    # Apply symbol mapping
)

# Add static backbone names as labels on the figure
for i, row in aggregated_metrics.iterrows():
    fig.add_annotation(
        x=row['Mean_Clean_Acc'] - 0.01,
        y=row['Mean_Robust_Acc'] + 0.01,  # Offset text slightly above the marker
        text=row['backbone'],
        showarrow=False,
        font=dict(size=10),
        align='center'
    )

# Add Pareto front markers with larger size and same shape/color as the original
for idx, d in enumerate(pareto_points.iterrows()):
    _, row = d
    fig.add_scatter(
        x=[row['Mean_Clean_Acc']],
        y=[row['Mean_Robust_Acc']],
        mode='markers',
        showlegend=False,  # Avoid duplicating legends
        marker=dict(
            size=13,  # Larger marker size
            symbol=symbol_mapping[row['loss_function']],  # Use the mapped symbol
            color=aggregated_metrics.loc[aggregated_metrics['backbone'] == row['backbone'], 'Architecture'].map({
                'ConvNext': '#1f77b4',  # Blue for ConvNext
                'ViT/DeiT': '#ff7f0e',  # Orange for ViT/DeiT
                'Other': '#2ca02c'     # Green for Other architectures
            }).values[0]  # Map the color dynamically
        ),
        name=f"Pareto"
    )


# Add a diagonal line (x = y)
fig.add_shape(
    type="line",
    x0=0, y0=0,  # Start of line
    x1=1, y1=1,  # End of line
    xref="paper", yref="paper",  # Use normalized coordinates (0 to 1 in plot space)
    line=dict(color="red", width=2, dash="dot")  # Line style
)

# Adjust layout for better visual clarity
fig.update_layout(
    legend_title=dict(text='Backbone and Loss Function'),
    width=600,
    height=600,
    font=dict(size=12),
    #title_font=dict(size=14),
    margin=dict(l=10, r=45, t=10, b=10),
    plot_bgcolor='rgba(0,0,0,0)',
    showlegend=False,
    xaxis=dict(showgrid=True, gridcolor='lightgrey', zeroline=False),
    yaxis=dict(showgrid=True, gridcolor='lightgrey', zeroline=False),
    yaxis_range=[0,0.7],
    xaxis_range=[0,0.85]
)


fig.update_layout(
    xaxis=dict(
        tickmode="array",
        tickvals=[0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85 ],
        ticktext=[0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85 ]
    ),
    yaxis=dict(
        tickmode="array",
        tickvals=[0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7 ],
        ticktext=[0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7 ]
    )
)

fig.show()
fig.write_image( "./figures/{}_{}.png".format(project_name,name) )